In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import cv2
import gc
from tqdm import tqdm
%matplotlib inline

import glob
import os

##
%matplotlib inline 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import json
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14
import seaborn as sns
import cv2 
import pandas as pd
import numpy as np
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense, Dropout, Flatten, Activation
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy,categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
start = dt.datetime.now()

In [2]:
## No of classes

In [2]:
print(len(os.listdir('./data/train_simplified')))
print(len(os.listdir('./data/train_raw')))

340
340


In [3]:
csv_files = r'D:\Project data\quickdraw-doodle-recognition\quickdraw_data\My code\data\train_simplified'
split_data = r'D:\Project data\quickdraw-doodle-recognition\quickdraw_data\My code\data\30K'

In [4]:
### I dont have the too much of GPU to process such large dataset 

In [5]:
nrows = 30000
files = os.listdir(csv_files)

In [6]:
## I intensionally commented the for loop since I dont want to execute it again

for file in tqdm(files, total=340):
    source_file_path = os.path.join(csv, file)
    df = pd.read_csv(source_file_path, usecols = ['countrycode', 'drawing', 'key_id', 'recognized'],
                     nrows = nrows)
    df1 = df.head(int(nrows/2))
    df1_train = df1.head(int(nrows/2/100*90))
    df1_valid = df1.tail(int(nrows/2/100*10))
    
    df2 = df.tail(int(nrows/2))
    df2_train = df2.head(int(nrows/2/100*90))
    df2_valid = df2.tail(int(nrows/2/100*10))
    
    path = os.path.join(split_data,'data_1', 'train')
    os.makedirs(path, exist_ok = True)
    df1_train.to_csv(os.path.join(path, file), index= False)
    path = os.path.join(split_data,'data_1', 'valid')
    os.makedirs(path, exist_ok = True)    
    df1_valid.to_csv(os.path.join(path, file), index= False)
    
    
    path = os.path.join(split_data,'data_2', 'train')
    os.makedirs(path, exist_ok = True)
    df2_train.to_csv(os.path.join(path, file), index= False)
    path = os.path.join(split_data,'data_2', 'valid')
    os.makedirs(path, exist_ok = True)   
    df2_valid.to_csv(os.path.join(path, file), index= False)
    

##Creating a subset of data with 30K datapoints in each class

#####Commented the writing to csv files below since I dont want to execute it again and again
for file in tqdm(files, total=340):
    source_file_path = os.path.join(csv_files, file)
    df = pd.read_csv(source_file_path, usecols = ['countrycode', 'drawing', 'key_id', 'recognized'],
                     nrows = nrows)
    df1 = df.head(int(nrows/2))
    df1_train = df1.head(int(nrows/2/100*90))
    df1_valid = df1.tail(int(nrows/2/100*10))
    
    df2 = df.tail(int(nrows/2))
    df2_train = df2.head(int(nrows/2/100*90))
    df2_valid = df2.tail(int(nrows/2/100*10))
    
    df_train = pd.concat([df1_train, df2_train])
    df_valid = pd.concat([df1_valid, df2_valid])
    
    path = os.path.join(split_data, 'train')
    os.makedirs(path, exist_ok = True)
    #df_train.to_csv(os.path.join(path, file), index= False)
    path = os.path.join(split_data,'valid')
    os.makedirs(path, exist_ok = True)    
    #df_valid.to_csv(os.path.join(path, file), index= False)
    

In [9]:
##files = glob.glob(r'D:\Project data\quickdraw-doodle-recognition\quickdraw_data\My code\data\30K\data_1\train\*.csv')
##len(files)

In [37]:
DP_DIR = r'./data/shuffle-csvs/'
##DP_DIR = r'./data/30K'
INPUT_DIR = './data/'
##INPUT_DIR = r'D:\Project data\quickdraw-doodle-recognition\quickdraw_data\My code\data\30K'

BASE_SIZE = 256
NCSVS = 100
NCATS = 340

np.random.seed(seed=1987)
tf.set_random_seed(seed=1987)

def f2cat(filename: str) -> str:
    return filename.split('.')[0]

#def list_all_categories():
#    files = os.listdir(os.path.join(INPUT_DIR, 'train_simplified'))
#    return ([f2cat(f) for f in files])

class Simplified():
    def __init__(self, input_path = './data'):
        self.input_path = input_path
    
    def list_all_categoris(self):
        files = os.listdir(os.path.join(self.input_path, 'train_simplified'))
        return ([f2cat(f) for f in files])
    
    def read_training_csv(self, category, nrows= None, ncols = None, drawing_transform = False):
        
        df = pd.read_csv(os.path.join(self.input_path, 'train_simplified',category+ '.csv' ), 
                         nrows= nrows,usecols = ncols)
        if drawing_transform:
            df['drawing'] = df['drawing'].apply(json.loads)
        
        return df


In [38]:
s = Simplified('./data')
NCSVS = 100
categories = s.list_all_categoris()
print(len(categories))

340


In [41]:
nrows =30000
##path = r'D:\Project data\quickdraw-doodle-recognition\quickdraw_data\My code\data\30K'
path = r'D:\Project data\quickdraw-doodle-recognition\quickdraw_data\My code\data\30K1111'
os.makedirs(path, exist_ok = True)

for y, category in tqdm(enumerate(categories)):
    df = s.read_training_csv(category, nrows= nrows)
    df['y'] = y
    df['cv'] = (df.key_id // 10 ** 7) % NCSVS
    for k in range(NCSVS):
        filename = 'train_k{}.csv'.format(k)
        chunk = df[df['cv']== k]
        chunk = chunk.drop(['cv'], axis =1)
        if y == 0:
            chunk.to_csv(os.path.join(path,filename), index = False)
        else:
            chunk.to_csv(os.path.join(path,filename),mode = 'a', header = False, index = False)


0it [00:00, ?it/s]
1it [00:02,  2.73s/it]
2it [00:05,  2.86s/it]
3it [00:09,  2.98s/it]
4it [00:12,  3.10s/it]
5it [00:15,  3.09s/it]
6it [00:18,  3.14s/it]
7it [00:21,  3.07s/it]
8it [00:24,  3.10s/it]
9it [00:28,  3.10s/it]
10it [00:31,  3.17s/it]
11it [00:34,  3.12s/it]
12it [00:38,  3.36s/it]
13it [00:41,  3.18s/it]
14it [00:44,  3.22s/it]
15it [00:47,  3.20s/it]
16it [00:50,  3.11s/it]
17it [00:53,  3.23s/it]
18it [00:57,  3.33s/it]
19it [01:01,  3.42s/it]
20it [01:04,  3.43s/it]
21it [01:07,  3.33s/it]
22it [01:11,  3.42s/it]
23it [01:14,  3.49s/it]
24it [01:20,  4.00s/it]
25it [01:23,  3.78s/it]
26it [01:27,  3.87s/it]
27it [01:30,  3.71s/it]
28it [01:33,  3.41s/it]
29it [01:37,  3.59s/it]
30it [01:40,  3.50s/it]
31it [01:44,  3.65s/it]
32it [01:48,  3.59s/it]
33it [01:53,  4.07s/it]
34it [01:57,  4.00s/it]
35it [02:00,  3.78s/it]
36it [02:03,  3.42s/it]
37it [02:06,  3.39s/it]
38it [02:09,  3.21s/it]
39it [02:13,  3.52s/it]
40it [02:18,  4.00s/it]
41it [02:21,  3.65s/it]
42it 

332it [13:02,  2.01s/it]
333it [13:05,  2.23s/it]
334it [13:07,  2.11s/it]
335it [13:08,  1.82s/it]
336it [13:09,  1.68s/it]
337it [13:12,  1.91s/it]
338it [13:14,  1.94s/it]
339it [13:19,  2.98s/it]
340it [13:20,  2.43s/it]

In [42]:
from sklearn.utils import shuffle
path = r'D:\Project data\quickdraw-doodle-recognition\quickdraw_data\My code\data\30K'
path_new = r'D:\Project data\quickdraw-doodle-recognition\quickdraw_data\My code\data\30K_zip1111'
os.makedirs(path_new, exist_ok = True)

for k in tqdm(range(NCSVS)):
    filename = 'train_k{}.csv'.format(k)
    df = pd.read_csv(os.path.join(path,filename))
    df = shuffle(df)
    ##Uncomment to populate .gz file
    ##df.to_csv(os.path.join(path_new,filename + '.gz'), compression='gzip',index = False)


 67%|██████████████████████████████████████████████████████▎                          | 67/100 [32:29<15:17, 27.81s/it]


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [47:55<00:00, 28.73s/it]

In [12]:


def apk(actual, predicted, k=3):
    """

    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):

        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

In [14]:
apk(actual= [1,2,3], predicted = [2,1,3])

1.0

In [77]:
def mapk(actual, predicted, k=3):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual,predicted)])

In [15]:
def preds2catids(predictions):
    return pd.DataFrame(np.argsort(-predictions, axis=1)[:, :3], columns=['a', 'b', 'c'])

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

In [16]:
STEPS = 800
EPOCHS = 16
size = 64
batchsize = 680

In [17]:
model = MobileNet(input_shape = (size, size,1), alpha = 1.0, weights = None, classes = NCATS)

model.compile(optimizer = Adam(lr = .002), loss = 'categorical_crossentropy' , 
              metrics = [ categorical_crossentropy,categorical_accuracy,top_3_accuracy])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 65, 65, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 32, 32, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 32, 32, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 32, 32, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 32, 32, 32)        128       
__________

_________________________________________________________________
conv_pw_9_relu (ReLU)        (None, 4, 4, 512)         0         
_________________________________________________________________
conv_dw_10 (DepthwiseConv2D) (None, 4, 4, 512)         4608      
_________________________________________________________________
conv_dw_10_bn (BatchNormaliz (None, 4, 4, 512)         2048      
_________________________________________________________________
conv_dw_10_relu (ReLU)       (None, 4, 4, 512)         0         
_________________________________________________________________
conv_pw_10 (Conv2D)          (None, 4, 4, 512)         262144    
_________________________________________________________________
conv_pw_10_bn (BatchNormaliz (None, 4, 4, 512)         2048      
_________________________________________________________________
conv_pw_10_relu (ReLU)       (None, 4, 4, 512)         0         
_________________________________________________________________
conv_dw_11

Training with Image Generator

In [18]:
## Drawing Grey Scale Images
def draw_cv2(raw_strokes, size = 64, lw =6, time_color = True):
    img_size = np.zeros((BASE_SIZE,BASE_SIZE), np.uint8)
    for t,stroke in enumerate(raw_strokes):
        for i in range(len(strokes[0]) - 1):
            if time_color:
                color = 255 - min(10,t)*13
            else:
                color = 255
            _ = cv2.line(img, (stroke[0][i],stroke[1][i]),
                        (stroke[0][i+1],stroke[1][i+1]),color, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size,size))
    else:
        return img


## Drawing RGB Images
## def draw_cv2_color later on 

In [23]:
def image_generator_xd(size, batchsize, ks, lw=6, time_color=True):
    while True:
        for k in np.random.permutation(ks):
            filename = os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(k))
            for df in pd.read_csv(filename, chunksize=batchsize):
                df['drawing'] = df['drawing'].apply(json.loads)
                x = np.zeros((len(df), size, size, 1))
                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw,
                                             time_color=time_color)
                x = preprocess_input(x).astype(np.float32)
                y = keras.utils.to_categorical(df.y, num_classes=NCATS)
                yield x, y

                
def df_to_image_array_xd(df, size, lw=6, time_color=True):
    df['drawing'] = df['drawing'].apply(json.loads)
    x = np.zeros((len(df), size, size, 1))
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw, time_color=time_color)
    x = preprocess_input(x).astype(np.float32)
    return x

In [24]:


valid_df = pd.read_csv(os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(NCSVS - 1)), nrows=34000)
x_valid = df_to_image_array_xd(valid_df, size)
y_valid = keras.utils.to_categorical(valid_df.y, num_classes=NCATS)
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))



FileNotFoundError: [Errno 2] No such file or directory: './data/shuffle-csvs/train_k99.csv.gz'

In [25]:
def f2cat(filename: str) -> str:
    return filename.split('.')[0]

class Simplified():
    def __init__(self, input_path='./data'):
        self.input_path = input_path

    def list_all_categories(self):
        files = os.listdir(os.path.join(self.input_path, 'train_simplified'))
        return sorted([f2cat(f) for f in files], key=str.lower)

    def read_training_csv(self, category, nrows=None, usecols=None, drawing_transform=False):
        df = pd.read_csv(os.path.join(self.input_path, 'train_simplified', category + '.csv'),
                         nrows=nrows, parse_dates=['timestamp'], usecols=usecols)
        if drawing_transform:
            df['drawing'] = df['drawing'].apply(json.loads)
        return df

340
